In [12]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101, NASNetMobile, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

tf.config.optimizer.set_jit(False)

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    try:
        tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError:
        pass

tf.keras.mixed_precision.set_global_policy('mixed_float16')

AUTOTUNE = tf.data.AUTOTUNE

DATA_DIR   = r'C:/Users/maxim/Documents/DLProjectDress/datasets'

BATCH_SIZE = 32
EPOCHS     = 5
LR         = 1e-4
INPUT_SIZE = (224, 224)

# compute global mean & std of the 'year' label
all_dfs = [pd.read_csv(os.path.join(DATA_DIR, f'fold{i}.csv'))
           for i in range(10)]
df_all  = pd.concat(all_dfs, ignore_index=True)
YEAR_MEAN = df_all['year'].mean()
YEAR_STD  = df_all['year'].std()
print(f'Label mean year = {YEAR_MEAN:.2f}, std = {YEAR_STD:.2f}')


Label mean year = 1845.86, std = 16.43


In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
def preprocess(path, label, training):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, INPUT_SIZE)
    if training:
        img = tf.image.random_flip_left_right(img)
    img = img / 255.0

    # normalize the year label to zero‐mean, unit‐std
    label = (label - YEAR_MEAN) / YEAR_STD
    return img, tf.cast(label, tf.float32)

def make_dataset(df, training):
    df = df.copy()
    #df['file'] = df['file'].str.replace(r'^datasets[\\/]', '', regex=True)

    files  = [os.path.join(DATA_DIR, p) for p in df['file']]
    labels = df['year'].values

    ds = tf.data.Dataset.from_tensor_slices((files, labels))
    if training:
        ds = ds.shuffle(buffer_size=len(files))
    ds = ds.map(lambda x,y: preprocess(x,y,training),
                num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds


In [15]:
def build_model(name, base, input_shape=(*INPUT_SIZE, 3), lr=LR):
    x = GlobalAveragePooling2D()(base.output)
    out = Dense(1)(x)   # regression head

    model = Model(base.input, out)
    print(model.summary())
    model.compile(optimizer=Adam(lr), loss='mse', metrics=['mae'])
    return model

In [ ]:
models = ['resnet101','nasnetmobile','inceptionv3']
results = {m: [] for m in models}

for model_name in models:
    if model_name == 'resnet101':
        base = ResNet101(include_top=False, weights='imagenet',
                         input_shape=(*INPUT_SIZE, 3))
    elif model_name == 'nasnetmobile':
        base = NASNetMobile(include_top=False, weights='imagenet',
                            input_shape=(*INPUT_SIZE, 3))
    elif model_name == 'inceptionv3':
        base = InceptionV3(include_top=False, weights='imagenet',
                           input_shape=(*INPUT_SIZE, 3))
    else:
        raise ValueError(f'Unknown model: {model_name}')
    
    print(base.summary())
    print(f'\n=== {model_name} ===')
    for fold in range(10):
        # Load the CSVs from DATA_DIR
        val_df   = pd.read_csv(os.path.join(DATA_DIR, f'fold{fold}.csv'))
        train_df = pd.concat([
            pd.read_csv(os.path.join(DATA_DIR, f'fold{i}.csv'))
            for i in range(10) if i != fold
        ], ignore_index=True)

        # Strip leading 'datasets/' so paths are RELATIVE to DATA_DIR
        for df in (train_df, val_df):
            df['file'] = df['file'].str.replace(
                r'^datasets[\\/]', '', regex=True
            )

        # Build tf.data pipelines
        train_ds = make_dataset(train_df, training=True)
        val_ds   = make_dataset(val_df,   training=False)

        # Train & checkpoint
        model = build_model(model_name, base)
        ckpt = tf.keras.callbacks.ModelCheckpoint(
            f'{model_name}_fold{fold}.h5',
            monitor='val_loss', save_best_only=True
        )
        

Model: "resnet101"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

In [ ]:
models = ['resnet101','nasnetmobile','inceptionv3']
results = {m: [] for m in models}

for model_name in models:
    print(f'\n=== {model_name} ===')
    for fold in range(10):
        # Load the CSVs from DATA_DIR
        val_df   = pd.read_csv(os.path.join(DATA_DIR, f'fold{fold}.csv'))
        train_df = pd.concat([
            pd.read_csv(os.path.join(DATA_DIR, f'fold{i}.csv'))
            for i in range(10) if i != fold
        ], ignore_index=True)

        # Strip leading 'datasets/' so paths are RELATIVE to DATA_DIR
        for df in (train_df, val_df):
            df['file'] = df['file'].str.replace(
                r'^datasets[\\/]', '', regex=True
            )

        # Build tf.data pipelines
        train_ds = make_dataset(train_df, training=True)
        val_ds   = make_dataset(val_df,   training=False)

        # Train & checkpoint
        model = build_model(model_name)
        ckpt = tf.keras.callbacks.ModelCheckpoint(
            f'{model_name}_fold{fold}.h5',
            monitor='val_loss', save_best_only=True
        )
        hist = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=EPOCHS,
            callbacks=[ckpt],
            verbose=2
        )

        best = min(hist.history['val_loss'])
        results[model_name].append(best)
        print(f' Fold {fold} → best val_mse: {best:.4f}')

print('\nCV MSE per model:')
for m, vals in results.items():
    print(f' {m}:', [f'{v:.4f}' for v in vals])

pd.DataFrame(results).to_csv('cv_results.csv', index=False)



=== resnet101 ===
Epoch 1/5
354/354 - 69s - loss: 0.3030 - mae: 0.4158 - val_loss: 1.1235 - val_mae: 0.8942


c:\Users\maxim\anaconda3\envs\deep_learning_env\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/5
354/354 - 52s - loss: 0.1065 - mae: 0.2489 - val_loss: 0.5241 - val_mae: 0.5791
Epoch 3/5
354/354 - 51s - loss: 0.0602 - mae: 0.1869 - val_loss: 0.2199 - val_mae: 0.3517
Epoch 4/5
354/354 - 53s - loss: 0.0420 - mae: 0.1543 - val_loss: 0.0814 - val_mae: 0.1921
Epoch 5/5
354/354 - 52s - loss: 0.0343 - mae: 0.1391 - val_loss: 0.0758 - val_mae: 0.1860
 Fold 0 → best val_mse: 0.0758
Epoch 1/5
354/354 - 62s - loss: 0.3625 - mae: 0.4466 - val_loss: 1.0310 - val_mae: 0.8876


c:\Users\maxim\anaconda3\envs\deep_learning_env\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/5
354/354 - 51s - loss: 0.1108 - mae: 0.2542 - val_loss: 0.4861 - val_mae: 0.5557
Epoch 3/5
354/354 - 51s - loss: 0.0623 - mae: 0.1895 - val_loss: 0.1719 - val_mae: 0.3004
Epoch 4/5
354/354 - 51s - loss: 0.0476 - mae: 0.1632 - val_loss: 0.1048 - val_mae: 0.2373
Epoch 5/5
354/354 - 51s - loss: 0.0373 - mae: 0.1446 - val_loss: 0.0969 - val_mae: 0.2097
 Fold 1 → best val_mse: 0.0969
Epoch 1/5
354/354 - 64s - loss: 0.3170 - mae: 0.4295 - val_loss: 1.0441 - val_mae: 0.9052


...

c:\Users\maxim\anaconda3\envs\deep_learning_env\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/5
354/354 - 53s - loss: 0.1010 - mae: 0.2414 - val_loss: 0.4860 - val_mae: 0.5509
Epoch 3/5
354/354 - 54s - loss: 0.0569 - mae: 0.1789 - val_loss: 0.2479 - val_mae: 0.3593
Epoch 4/5
354/354 - 54s - loss: 0.0418 - mae: 0.1527 - val_loss: 0.0828 - val_mae: 0.2082
Epoch 5/5
354/354 - 53s - loss: 0.0345 - mae: 0.1389 - val_loss: 0.0819 - val_mae: 0.1997
 Fold 9 → best val_mse: 0.0819

=== nasnetmobile ===
20004864/19993432 [==============================] - 1s 0us/step
Epoch 1/5
354/354 - 71s - loss: 0.2927 - mae: 0.3996 - val_loss: 0.6796 - val_mae: 0.6915


c:\Users\maxim\anaconda3\envs\deep_learning_env\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/5
354/354 - 42s - loss: 0.1115 - mae: 0.2450 - val_loss: 0.4110 - val_mae: 0.5245
Epoch 3/5


KeyboardInterrupt: 

In [ ]:
# Only run the remaining models
models = ['nasnetmobile','inceptionv3']
results_other = {m: [] for m in models}

for model_name in models:
    print(f'\n=== {model_name} ===')
    for fold in range(10):
        val_df = pd.read_csv(os.path.join(DATA_DIR, f'fold{fold}.csv'))
        train_df = pd.concat([
            pd.read_csv(os.path.join(DATA_DIR, f'fold{i}.csv'))
            for i in range(10) if i != fold
        ], ignore_index=True)

        # build tf.data pipelines (with normalization baked in)
        train_ds = make_dataset(train_df, training=True)
        val_ds   = make_dataset(val_df,   training=False)

        model = build_model(model_name)
        ckpt = tf.keras.callbacks.ModelCheckpoint(
            f'{model_name}_fold{fold}.h5',
            monitor='val_loss', save_best_only=True
        )
        hist = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=EPOCHS,
            callbacks=[ckpt],
            verbose=2
        )

        best = min(hist.history['val_loss'])
        results_other[model_name].append(best)
        print(f' Fold {fold} → best normalized-MSE: {best:.4f}')

print('\nCV normalized-MSE for remaining models:')
for m, vals in results_other.items():
    print(f' {m}:', [f'{v:.4f}' for v in vals])



=== nasnetmobile ===
Epoch 1/5
354/354 - 79s - loss: 0.2656 - mae: 0.3762 - val_loss: 0.6058 - val_mae: 0.6201


c:\Users\maxim\anaconda3\envs\deep_learning_env\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/5
354/354 - 46s - loss: 0.1063 - mae: 0.2361 - val_loss: 0.3450 - val_mae: 0.4547
Epoch 3/5
354/354 - 46s - loss: 0.0654 - mae: 0.1865 - val_loss: 0.2753 - val_mae: 0.3887
Epoch 4/5
354/354 - 46s - loss: 0.0485 - mae: 0.1630 - val_loss: 0.1995 - val_mae: 0.3276
Epoch 5/5
354/354 - 46s - loss: 0.0372 - mae: 0.1444 - val_loss: 0.2323 - val_mae: 0.3513
 Fold 0 → best normalized-MSE: 0.1995
Epoch 1/5
354/354 - 71s - loss: 0.2906 - mae: 0.3935 - val_loss: 0.5012 - val_mae: 0.5421


c:\Users\maxim\anaconda3\envs\deep_learning_env\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/5
354/354 - 45s - loss: 0.1134 - mae: 0.2441 - val_loss: 0.3946 - val_mae: 0.4710
Epoch 3/5
354/354 - 45s - loss: 0.0730 - mae: 0.1997 - val_loss: 0.3083 - val_mae: 0.4120
Epoch 4/5
354/354 - 45s - loss: 0.0521 - mae: 0.1694 - val_loss: 0.2653 - val_mae: 0.3833
Epoch 5/5
354/354 - 45s - loss: 0.0411 - mae: 0.1506 - val_loss: 0.1995 - val_mae: 0.3360
 Fold 1 → best normalized-MSE: 0.1995
Epoch 1/5
354/354 - 74s - loss: 0.2694 - mae: 0.3783 - val_loss: 0.5720 - val_mae: 0.5720


...

Epoch 2/5
354/354 - 46s - loss: 0.1112 - mae: 0.2439 - val_loss: 0.2555 - val_mae: 0.3896
Epoch 3/5
354/354 - 46s - loss: 0.0720 - mae: 0.1974 - val_loss: 0.2315 - val_mae: 0.3707
Epoch 4/5
354/354 - 46s - loss: 0.0502 - mae: 0.1664 - val_loss: 0.2370 - val_mae: 0.3719
Epoch 5/5
354/354 - 47s - loss: 0.0415 - mae: 0.1523 - val_loss: 0.2286 - val_mae: 0.3574
 Fold 7 → best normalized-MSE: 0.2286
Epoch 1/5
354/354 - 74s - loss: 0.2782 - mae: 0.3828 - val_loss: 0.7233 - val_mae: 0.7080


c:\Users\maxim\anaconda3\envs\deep_learning_env\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/5
354/354 - 47s - loss: 0.1096 - mae: 0.2425 - val_loss: 0.3435 - val_mae: 0.4588
Epoch 3/5
354/354 - 47s - loss: 0.0719 - mae: 0.1965 - val_loss: 0.3781 - val_mae: 0.4860
Epoch 4/5
354/354 - 47s - loss: 0.0519 - mae: 0.1692 - val_loss: 0.2823 - val_mae: 0.4101
Epoch 5/5
354/354 - 47s - loss: 0.0400 - mae: 0.1482 - val_loss: 0.2266 - val_mae: 0.3604
 Fold 8 → best normalized-MSE: 0.2266
Epoch 1/5
354/354 - 74s - loss: 0.2773 - mae: 0.3869 - val_loss: 0.9937 - val_mae: 0.8631


c:\Users\maxim\anaconda3\envs\deep_learning_env\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/5
354/354 - 46s - loss: 0.1110 - mae: 0.2427 - val_loss: 0.4786 - val_mae: 0.5696
Epoch 3/5
354/354 - 46s - loss: 0.0716 - mae: 0.1975 - val_loss: 0.2831 - val_mae: 0.3997
Epoch 4/5
354/354 - 47s - loss: 0.0515 - mae: 0.1684 - val_loss: 0.2899 - val_mae: 0.4171
Epoch 5/5
354/354 - 46s - loss: 0.0390 - mae: 0.1475 - val_loss: 0.2571 - val_mae: 0.3840
 Fold 9 → best normalized-MSE: 0.2571

=== inceptionv3 ===
87924736/87910968 [==============================] - 3s 0us/step
Epoch 1/5
354/354 - 45s - loss: 0.2594 - mae: 0.3766 - val_loss: 0.1267 - val_mae: 0.2540
Epoch 2/5
354/354 - 31s - loss: 0.1147 - mae: 0.2531 - val_loss: 0.1401 - val_mae: 0.2765
Epoch 3/5
354/354 - 31s - loss: 0.0775 - mae: 0.2087 - val_loss: 0.0816 - val_mae: 0.1984
Epoch 4/5
354/354 - 31s - loss: 0.0539 - mae: 0.1727 - val_loss: 0.0862 - val_mae: 0.2054
Epoch 5/5
354/354 - 31s - loss: 0.0405 - mae: 0.1509 - val_loss: 0.0953 - val_mae: 0.2106
 Fold 0 → best normalized-MSE: 0.0816
Epoch 1/5
354/354 - 39s - lo

In [ ]:
# InceptionV3 across all 10 folds
model_name = 'inceptionv3'
results_inc = []

for fold in range(10):
    print(f'\n=== {model_name} | Fold {fold} ===')
    # load CSVs
    val_df = pd.read_csv(os.path.join(DATA_DIR, f'fold{fold}.csv'))
    train_df = pd.concat([
        pd.read_csv(os.path.join(DATA_DIR, f'fold{i}.csv'))
        for i in range(10) if i != fold
    ], ignore_index=True)

    # build tf.data pipelines (with your normalization baked in)
    train_ds = make_dataset(train_df, training=True)
    val_ds   = make_dataset(val_df,   training=False)

    # build, train & checkpoint
    model = build_model(model_name)
    ckpt = tf.keras.callbacks.ModelCheckpoint(
        f'{model_name}_fold{fold}.h5',
        monitor='val_loss', save_best_only=True
    )
    hist = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[ckpt],
        verbose=2
    )

    # capture best normalized-MSE
    best = min(hist.history['val_loss'])
    results_inc.append(best)
    print(f'→ best normalized-MSE: {best:.4f}')

# Final summary
import numpy as np
mean_mse = np.mean(results_inc)
mean_rmse = np.sqrt(mean_mse)
print(f'\nInceptionV3 10-fold mean MSEₙ: {mean_mse:.4f}')
print(f'InceptionV3 10-fold mean RMSEₙ: {mean_rmse:.4f}')
print('≈', mean_rmse * YEAR_STD, 'years off on average')



=== inceptionv3 | Fold 0 ===
Epoch 1/5
354/354 - 46s - loss: 0.2670 - mae: 0.3839 - val_loss: 0.1522 - val_mae: 0.2834
Epoch 2/5
354/354 - 30s - loss: 0.1178 - mae: 0.2565 - val_loss: 0.1325 - val_mae: 0.2768
Epoch 3/5
354/354 - 30s - loss: 0.0778 - mae: 0.2102 - val_loss: 0.0868 - val_mae: 0.1975
Epoch 4/5
354/354 - 30s - loss: 0.0587 - mae: 0.1817 - val_loss: 0.0841 - val_mae: 0.1920
Epoch 5/5
354/354 - 30s - loss: 0.0430 - mae: 0.1548 - val_loss: 0.0712 - val_mae: 0.1797
→ best normalized-MSE: 0.0712

=== inceptionv3 | Fold 1 ===
Epoch 1/5
354/354 - 36s - loss: 0.2687 - mae: 0.3874 - val_loss: 0.1391 - val_mae: 0.2693
Epoch 2/5
354/354 - 30s - loss: 0.1158 - mae: 0.2527 - val_loss: 0.1026 - val_mae: 0.2212
Epoch 3/5
354/354 - 30s - loss: 0.0703 - mae: 0.1983 - val_loss: 0.0966 - val_mae: 0.2173
Epoch 4/5
354/354 - 30s - loss: 0.0544 - mae: 0.1758 - val_loss: 0.1020 - val_mae: 0.2239
Epoch 5/5
354/354 - 30s - loss: 0.0416 - mae: 0.1519 - val_loss: 0.0757 - val_mae: 0.1851
→ best nor

In [ ]:
import numpy as np
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# 0) Load & freeze the pretrained base just once
frozen_base = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(*INPUT_SIZE, 3)
)
frozen_base.trainable = False

# 1) Run 10-fold CV on only this frozen model
model_name = 'inceptionv3_frozen'
results_inc = []

for fold in range(10):
    print(f'\n=== {model_name} | Fold {fold} ===')
    # ─── Load CSVs ───
    val_df   = pd.read_csv(os.path.join(DATA_DIR, f'fold{fold}.csv'))
    train_df = pd.concat([
        pd.read_csv(os.path.join(DATA_DIR, f'fold{i}.csv'))
        for i in range(10) if i != fold
    ], ignore_index=True)

    # ─── Strip everything up through the 'datasets/' folder ───
    # This handles both forward and backslashes.
    for df in (train_df, val_df):
        df['file'] = df['file'].str.replace(
            r'.*datasets[\\/]', '', regex=True
        )

    # ─── Build datasets ───
    train_ds = make_dataset(train_df, training=True)
    val_ds   = make_dataset(val_df,   training=False)

    # ─── Attach a fresh regression head ───
    x   = GlobalAveragePooling2D()(frozen_base.output)
    out = Dense(1)(x)
    model = Model(frozen_base.input, out)
    model.compile(
        optimizer=Adam(LR),
        loss='mse',
        metrics=['mae']
    )

    # ─── Train & checkpoint ───
    ckpt = tf.keras.callbacks.ModelCheckpoint(
        f'{model_name}_fold{fold}.h5',
        monitor='val_loss',
        save_best_only=True
    )
    hist = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[ckpt],
        verbose=2
    )

    # ─── Record best normalized‐MSE ───
    best = min(hist.history['val_loss'])
    results_inc.append(best)
    print(f'→ best normalized-MSE: {best:.4f}')

# 2) Summarize
mean_mse  = np.mean(results_inc)
mean_rmse = np.sqrt(mean_mse)
print(f'\nFrozen-InceptionV3 10-fold mean MSEₙ: {mean_mse:.4f}')
print(f'Frozen-InceptionV3 10-fold mean RMSEₙ: {mean_rmse:.4f}')
print('≈', mean_rmse * YEAR_STD, 'years off on average')



=== inceptionv3_frozen | Fold 0 ===
DATA_DIR = C:/Users/maxim/Documents/DLProjectDress/datasets
C:/Users/maxim/Documents/DLProjectDress/datasets\public/1840/1844_2667vna.jpg exists? True
C:/Users/maxim/Documents/DLProjectDress/datasets\private/1820/1824_035_Zrzut ekranu 2022-07-26 205129.png exists? True
C:/Users/maxim/Documents/DLProjectDress/datasets\public/1840/1845_59washington.jpg exists? True
C:/Users/maxim/Documents/DLProjectDress/datasets\public/1840/1845_61washington.jpg exists? True
C:/Users/maxim/Documents/DLProjectDress/datasets\public/1820/1823_2604vna.jpg exists? True
Epoch 1/5


NotFoundError: 2 root error(s) found.
  (0) Not found:  NewRandomAccessFile failed to Create/Open: C:/Users/maxim/Documents/DLProjectDress/datasets\datasets/private/1840/1841_920etsy.jpg : The system cannot find the path specified.
; No such process
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
  (1) Not found:  NewRandomAccessFile failed to Create/Open: C:/Users/maxim/Documents/DLProjectDress/datasets\datasets/private/1840/1841_920etsy.jpg : The system cannot find the path specified.
; No such process
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_2]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_120778]

Function call stack:
train_function -> train_function
